In [1]:
import pandas as pd
import numpy as np
from torch import nn
import torch
from torch import tensor
 
import multiprocessing
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import RandomSampler, SequentialSampler
import yaml
import pickle
import torch
import torch.nn as nn


class clf_dfm(nn.Module):
    
    def __init__(
        self,
        num_domains,          # number of feilds
        inp_emb_dimensions,   # emb dimension of each of the 
        fm_inp_dim = 64,
        dnn_layer_dimensions = [512,256]  #  FNN after concatenated embedding
    ):
        
        super(clf_dfm, self).__init__()
        
        self.dnn_fc = nn.ModuleList()                      # Fully connected layers
        self.dnn_num_layers = len(dnn_layer_dimensions)+1  # Number of fully connected layers
        
        for i in range(self.dnn_num_layers):    
            if i == 0: 
                inp_dim = inp_emb_dimensions * num_domains #  Concatenation
            if i == self.dnn_num_layers-1:
                op_dim = 1  # Classifier output
            else:
                op_dim = dnn_layer_dimensions[i] 
            self.dnn_fc.append( 
                nn.Linear(inp_dim, op_dim)
            )
            inp_dim = op_dim
            
        self.fc2 = nn.Linear(2,2,bias=False)         
        
        self.num_domains = num_domains
        self.xform_fm_1 = nn.ModuleList(
            [
                nn.Linear(inp_emb_dimensions,fm_inp_dim) 
             for _ in range(self.num_domains)
            ]
        )
        print(self.xform_fm_1)
        
        return
    
    
    def forward(self, input_x):
        '''
        input x should be of shape [Batch, num_domains, emb_vec]
        '''
        
        # ----- DNN ------- #
        x_dnn = input_x.view(-1,12)
        
        for i in range(self.dnn_num_layers):    
            x_dnn = self.dnn_fc[i](x_dnn)
        
        print(x_dnn.shape)
        # ----- FM --------- #
        
        fm_input = input_x
        # transform inner products with a MLP
        fm_input = torch.chunk(
            fm_input,
            self.num_domains,
            dim = 1
        )
        
        print(fm_input[0].shape, len(fm_input))
        x_fm_input = []
        
        for i in range(self.num_domains):
            x_fm_input.append(
                self.xform_fm_1[i]
                (fm_input[i]).squeeze(1) 
            )
        x_fm_input = torch.stack(
             x_fm_input, dim=2   
        )    
        print(x_fm_input.shape)      
        
        square_of_sum = torch.pow(torch.sum(x_fm_input, dim=1, keepdim=True), 2)
        sum_of_square = torch.sum(x_fm_input * x_fm_input, dim=1, keepdim=True)
        cross_term = square_of_sum - sum_of_square
        cross_term = 0.5 * torch.sum(cross_term, dim=2, keepdim=False)
        print(cross_term.shape)
        x2 = torch.cat ([x_dnn, cross_term],dim = 1)
        op = self.fc2(x2)
        print(op.shape)
        op = torch.nn.functional.tanh(op)
        op = torch.nn.functional.softmax (op)
        return op
    
        

In [13]:
model =  clf_dfm(
    num_domains = 3,
    inp_emb_dimensions = 4,
    fm_inp_dim= 8,
    dnn_layer_dimensions = [5,6]
)

ModuleList(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): Linear(in_features=4, out_features=8, bias=True)
  (2): Linear(in_features=4, out_features=8, bias=True)
)


In [14]:
x = torch.FloatTensor(np.random.random([10,3,4]))
model(x)

torch.Size([10, 1])
torch.Size([10, 1, 4]) 3
torch.Size([10, 8, 3])
torch.Size([10, 1])
torch.Size([10, 2])


/home/ddatta/anaconda3/envs/tmp_venv/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/ddatta/anaconda3/envs/tmp_venv/lib/python3.6/site-packages/ipykernel_launcher.py:87: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.4964, 0.5036],
        [0.4961, 0.5039],
        [0.4980, 0.5020],
        [0.4962, 0.5038],
        [0.4968, 0.5032],
        [0.4976, 0.5024],
        [0.4976, 0.5024],
        [0.4947, 0.5053],
        [0.4952, 0.5048],
        [0.4967, 0.5033]], grad_fn=<SoftmaxBackward>)

In [15]:
x1 = np.random.random([1,3,4])
fm_input = x1

square_of_sum = np.power(
    np.sum(fm_input, axis=1, keepdims=True), 
    2
)
sum_of_square = np.sum(fm_input * fm_input, axis=1, keepdims=True)
cross_term = square_of_sum - sum_of_square
cross_term = 0.5 * np.sum(cross_term, axis=2, keepdims=False)


In [16]:
fm_input

array([[[0.10728022, 0.04979349, 0.50967466, 0.12437399],
        [0.37922166, 0.51520702, 0.43378518, 0.4104865 ],
        [0.70945453, 0.46002874, 0.43952501, 0.06980309]]])

In [17]:
cross_term

array([[1.39555665]])

In [18]:
sum_of_square

array([[[0.65864384, 0.47954411, 0.64112008, 0.18884053]]])

In [71]:
x1[0][0]

array([0.19505893, 0.19231236, 0.94403774, 0.89043176])

In [72]:
r = 0

for i in range(3):
    for j in range(i+1,3):
        r += np.dot(x1[0][i],x1[0][j])
    

In [74]:
r

3.1558231612885437